In [ ]:
from PIL import Image
import glob
import random
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os

from IPython.display import clear_output

from style_transfer import run_style_tranfer, gen_mask, save_annotation

In [ ]:
def load_dataset(clean_folder, file_list):
    IMG_SIZE = 200
    dataset_base_name = clean_folder.split("/")[-1]
    noisy_dataset1 = dataset_base_name+"_noisy_A1"
    noisy_dataset2 = dataset_base_name+"_noisy_A2"
    noisy_dataset3 = dataset_base_name+"_noisy_B1"
    noisy_dataset4 = dataset_base_name+"_noisy_C1"
    noisy_dataset5 = dataset_base_name+"_noisy_C2"
    noisy_dataset6 = dataset_base_name+"_noisy_D1"
    
    dataset_list = []
    dataset_name_list = []
    
    for folder_ in glob.glob(clean_folder + "/**"):
        folder_ = folder_.replace("\\", "/")
        folder_number = folder_.split("/")[-1]
        
        if folder_number in file_list:
            print(folder_number)
            
            for piece_ in glob.glob( os.path.join(clean_folder,folder_number)+ "/*.tiff"):
                piece_ = piece_.replace("\\", "/")
                #print(piece_)
                
                clean_img = np.array(Image.open(piece_)) 
                assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                
                if np.std(clean_img)> 0.001:
                    
                    sample_list = []
                    sample_list.append(clean_img)
                    dataset_name_list.append(piece_)
                    
                    noisy_img1 = np.array(Image.open(piece_.replace(dataset_base_name, noisy_dataset1)))
                    assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                    sample_list.append(noisy_img1)
                    
                    noisy_img2 = np.array(Image.open(piece_.replace(dataset_base_name, noisy_dataset2)))
                    assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                    sample_list.append(noisy_img2)
                    
                    noisy_img3 = np.array(Image.open(piece_.replace(dataset_base_name, noisy_dataset3)))
                    assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                    sample_list.append(noisy_img3)
                    
                    noisy_img4 = np.array(Image.open(piece_.replace(dataset_base_name, noisy_dataset4)))
                    assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                    sample_list.append(noisy_img4)
                    
                    noisy_img5 = np.array(Image.open(piece_.replace(dataset_base_name, noisy_dataset5)))
                    assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                    sample_list.append(noisy_img5)
                    
                    noisy_img6 = np.array(Image.open(piece_.replace(dataset_base_name, noisy_dataset6)))
                    assert clean_img.shape == (IMG_SIZE, IMG_SIZE)
                    sample_list.append(noisy_img6)
                    
                    #plt.imshow( np.hstack((noisy_img1,noisy_img2,noisy_img3,noisy_img4,noisy_img5,noisy_img6)) , cmap="bwr", vmin=0, vmax=1)
                    #plt.axis('off')
                    #plt.pause(0.1)
                    
                    dataset_list.append(np.array(sample_list))

    return np.array(dataset_list), dataset_name_list
    
dataset_tensor, file_name_list = load_dataset("../../dataset/croped/train/Dataset_Sintetico_02", ["20","21","22","23","24","25","26","27","28","29","30","31","32","33"])

print(dataset_tensor.shape)

In [ ]:
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.0),
    ('conv3_1', 0.0),
    ('conv4_1', 0.0),
    ('conv5_1', 8.0)]

epochs = 200

num_saples = dataset_tensor.shape[0]

border = np.ones((200,5))

print(num_saples)
assert len(file_name_list) == num_saples

for i in range(0,num_saples):
    img_clean = dataset_tensor[i,0,:,:]
    list_noisy = []  
    list_noisy.append(dataset_tensor[i,1,:,:])
    list_noisy.append(dataset_tensor[i,2,:,:])
    list_noisy.append(dataset_tensor[i,3,:,:])
    list_noisy.append(dataset_tensor[i,4,:,:])
    list_noisy.append(dataset_tensor[i,5,:,:])
    list_noisy.append(dataset_tensor[i,6,:,:])
    
    
    for j in range(0,6):
        img_noisy = list_noisy[j]
        
        gen_filename = file_name_list[i].replace("train", "gen").replace(".tiff", "_"+str(j)+".tiff")
        
        if os.path.exists(gen_filename):
            print("pass ", gen_filename)
            continue
        
        gen_image = run_style_tranfer(STYLE_W=STYLE_LAYERS, content_image=img_clean, style_image=img_noisy, num_epochs=epochs, lr=2.0)
        
        gen_image = Image.fromarray(gen_image) 
        
        head, _ = os.path.split(gen_filename)
        if not os.path.exists(head):
            os.makedirs(head)
        
        gen_image.save(gen_filename)
    

    clear_output(wait=True)